In [41]:
import re
import csv
import requests
import time, os
import itertools

from collections import defaultdict
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [43]:
columns = ['data.economy.budget.date', 'data.economy.budget.expenditures.units',
           'data.economy.budget.expenditures.value', 'data.economy.budget.revenues.units',
           'data.economy.budget.revenues.value', 'data.economy.budget_surplus_or_deficit.date',
           'data.economy.budget_surplus_or_deficit.percent_of_gdp', 'data.economy.exports.partners.by_country',
           'data.economy.exports.partners.date', 'data.economy.gdp.composition.by_end_use.date',
           'data.economy.gdp.composition.by_end_use.end_uses.exports_of_goods_and_services.units',
           'data.economy.gdp.composition.by_end_use.end_uses.exports_of_goods_and_services.value',
           'data.economy.gdp.composition.by_end_use.end_uses.household_consumption.value',
           'data.economy.household_income_by_percentage_share.highest_ten_percent.units',
           'data.economy.household_income_by_percentage_share.highest_ten_percent.value',
           'data.economy.household_income_by_percentage_share.lowest_ten_percent.units',
           'data.economy.household_income_by_percentage_share.lowest_ten_percent.value',
           'data.economy.industrial_production_growth_rate.annual_percentage_increase', 
           'data.economy.industrial_production_growth_rate.date', 'data.energy.electricity.access.date',
           'data.energy.electricity.access.total_electrification.value', 
           'data.energy.electricity.by_source.fossil_fuels.date',
           'data.energy.electricity.by_source.fossil_fuels.percent', 
           'data.energy.electricity.by_source.hydroelectric_plants.date',
           'data.energy.electricity.by_source.hydroelectric_plants.percent', 
           'data.energy.electricity.by_source.nuclear_fuels.date', 
           'data.energy.electricity.by_source.nuclear_fuels.percent',
           'data.energy.electricity.by_source.other_renewable_sources.date', 
           'data.energy.electricity.by_source.other_renewable_sources.percent']

In [44]:
#credit: Roman Konoval StackOverflow
#https://stackoverflow.com/questions/43382447/python-with-selenium-drag-and-drop-from-file-system-to-webdriver

'''
Uses JS_DROP_FILE script to enable selenium to drag and drop a file from 
local machine as input for website.
'''

JS_DROP_FILE = """
    var target = arguments[0],
        offsetX = arguments[1],
        offsetY = arguments[2],
        document = target.ownerDocument || document,
        window = document.defaultView || window;

    var input = document.createElement('INPUT');
    input.type = 'file';
    input.onchange = function () {
      var rect = target.getBoundingClientRect(),
          x = rect.left + (offsetX || (rect.width >> 1)),
          y = rect.top + (offsetY || (rect.height >> 1)),
          dataTransfer = { files: this.files };

      ['dragenter', 'dragover', 'drop'].forEach(function (name) {
        var evt = document.createEvent('MouseEvent');
        evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
        evt.dataTransfer = dataTransfer;
        target.dispatchEvent(evt);
      });

      setTimeout(function () { document.body.removeChild(input); }, 25);
    };
    document.body.appendChild(input);
    return input;
"""

def drag_and_drop_file(drop_target, path):
    driver = drop_target.parent
    file_input = driver.execute_script(JS_DROP_FILE, drop_target, 0, 0)
    file_input.send_keys(path)

In [45]:
#credit to: Rob Hawkins from stackoverflow
#https://stackoverflow.com/questions/37979644/parse-beautifulsoup-element-into-selenium

def xpath_soup(element):
    """
    Generate xpath of soup element
    :param element: bs4 text or node
    :return: xpath as string
    """
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        """
        @type parent: bs4.element.Tag
        """
        previous = itertools.islice(parent.children, 0, parent.contents.index(child))
        xpath_tag = child.name
        xpath_index = sum(1 for i in previous if i.name == xpath_tag) + 1
        components.append(xpath_tag if xpath_index == 1 else '%s[%d]' % (xpath_tag, xpath_index))
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [46]:
def check_boxes(soup):
    '''
    Check all the boxes for features of interest (specified in 'columns').
    This method may result in some leak-over of features not explicitly specified in 'columns'
    list. This is due to the cases where the column is not one of the checkbox options
    
    check_boxes() will print out all features in 'columns' list that does not appear as a 
    check box option
    '''
    global columns
    for i in range(len(columns)):
        try:
            soup_element = soup.find(text=columns[i]).find_previous('input')
            xpath = xpath_soup(soup_element)
            driver.find_element_by_xpath(xpath).click()
        except:
            print('"{}" not found'.format(columns[i]))
            pass

In [54]:
def scrape(file_path):
    '''
    Uses Selenium web driver to automate interaction with website.
    - Upload factbook.json file
    - Click checkboxes according to 'columns' elements
    - Retrieves organized data
    
    Input: file path/file name
    Output: header - header row (list)
            data - list of dictionaries mapping header elements to data element
    '''
    chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
    os.environ["webdriver.chrome.driver"] = chromedriver

    url = 'https://iancoleman.io/explorer-cia-world-factbook/'
    
    driver = webdriver.Chrome()
    driver.get(url)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    chooseFile = driver.find_element_by_xpath('//div/p[@class="dz-clickable"]')
    
    path = os.path.abspath(file_path)
    drag_and_drop_file(chooseFile, path)
    time.sleep(1)
    
    change_columns_button = driver.find_element_by_xpath('//p/button[@type="button"]').click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    
    #check_boxes(soup) 
    ###for some reason, check_boxes() function is inoperable within scrape()
    ###BUT it works well when on its own... ???
    
    for i in range(len(columns)):
        try:
            soup_element = soup.find(text=columns[i]).find_previous('input')
            xpath = xpath_soup(soup_element)
            check_box = driver.find_element_by_xpath(xpath).click()
        except:
            print('"{}" not found'.format(columns[i]))
            pass
    
    x_button = driver.find_element_by_xpath('//div/button[@aria-label="Close"]').click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    driver.close()
    
    return get_header(soup), get_data(soup)
    

In [55]:
def get_header(soup):
    '''
    Retrieves header from soup
    
    Input: BeautfifulSoup.soup element
    Output: list of header (str) elements
    '''
    #regex credit: Brian Balzar from stackoverflow
    #https://stackoverflow.com/questions/53014806/using-regex-to-extract-from-second-period-to-end-of-a-string

    table_headers = soup.find_all('th')
    header = []
    for item in table_headers:
        string = item.text
        try:
            subject = re.match('^([^.]+)\.([^.]+)\.(.+)$', string).group(3)
            subject = subject.replace('.', '_')
        except:
            subject = string
        header.append(subject)
    return header

In [56]:
def get_data(soup):
    '''
    Retrieves table element
    Parses through rows and collects data
    
    Input: BeautifulSoup.soup element
    Output: list of dictionaries containing data
    '''
    data = []
    
    header = get_header(soup)
    table = soup.find('table')
    rows = [row for row in table.find_all('tr')][1:]

    for row in rows:
        row_data = defaultdict()
        for i in range(len(header)):
            row_data[header[i]] = row.find_all('td')[i].text
        data.append(row_data)
        
    return data

In [57]:
def write(file_name):
    '''
    Writes out scraped data as a csv file
    
    Input: file_ path/file name of factbook.json file
    Output: CSV
    '''
    keys, data = scrape(file_name)
    date = re.search('\/(([\d]*-)+[\d]*)', file_name).group(1)
    with open('{}_data.csv'.format(date), 'w', newline='') as write_obj:
        # create writer object
        dict_writer = csv.DictWriter(write_obj, fieldnames = keys)
        #dict_writer.writeheader()
        dict_writer.writerows(data)


In [1]:
file_name = '2019-12-30_factbook.json'

In [3]:
import re
re.search('\/(([\d]*-)+[\d]*)', file_name).group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [51]:
keys, data = scrape(file_name)

In [52]:
write(file_name)

In [53]:
#validate working status
import pandas as pd
pd.read_csv(write_out_file)

,World,2017,USD,"23,810,000,000,000",USD.1,"21,680,000,000,000",2016,-3,Unnamed: 8,Unnamed: 9,...,2015,63,2015.1,18,2015.2,6,2015.3,14,86.3,"7,684,292,383"
0,Afghanistan,2017.0,USD,"5,328,000,000",USD,"2,276,000,000",2017.0,-15.1,"[{""name"":""India"",""percent"":56.5},{""name"":""Paki...",2017.0,...,2016.0,45.0,2017.0,52.0,2017.0,0.0,2017.0,4.0,43.0,"36,643,815"
1,Akrotiri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"15,500"
2,Albania,2017.0,USD,"3,874,000,000",USD,"3,614,000,000",2017.0,-2.0,"[{""name"":""Italy"",""percent"":53.4},{""name"":""Koso...",2017.0,...,2016.0,5.0,2017.0,95.0,2017.0,0.0,2017.0,0.0,98.1,"3,074,579"
3,Algeria,2017.0,USD,"70,200,000,000",USD,"54,150,000,000",2017.0,-9.6,"[{""name"":""Italy"",""percent"":17.4},{""name"":""Spai...",2017.0,...,2016.0,96.0,2017.0,1.0,2017.0,0.0,2017.0,2.0,81.4,"42,972,878"
4,American Samoa,2016.0,USD,"262,500,000",USD,"249,000,000",2016.0,-2.1,"[{""name"":""Australia"",""percent"":25},{""name"":""Gh...",2017.0,...,2016.0,98.0,2017.0,0.0,2017.0,0.0,2017.0,2.0,NaN,"49,437"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016.0,100.0,2017.0,0.0,2017.0,0.0,2017.0,0.0,NaN,"652,271"
254,Yemen,2017.0,USD,"4,458,000,000",USD,"2,821,000,000",2017.0,-5.2,"[{""name"":""Egypt"",""percent"":29.4},{""name"":""Thai...",2017.0,...,2016.0,79.0,2017.0,0.0,2017.0,0.0,2017.0,21.0,70.1,"29,884,405"
255,Zambia,2017.0,USD,"6,357,000,000",USD,"4,473,000,000",2017.0,-7.3,"[{""name"":""Switzerland"",""percent"":44.8},{""name""...",2017.0,...,2016.0,5.0,2017.0,93.0,2017.0,0.0,2017.0,2.0,86.7,"17,426,623"
256,Zimbabwe,2017.0,USD,"5,500,000,000",USD,"3,800,000,000",2017.0,-9.6,"[{""name"":""South Africa"",""percent"":50.3},{""name...",2017.0,...,2016.0,58.0,2017.0,37.0,2017.0,0.0,2017.0,5.0,86.5,"14,546,314"
